# Task

Despite being a binary classification problem, problem 2 wants us to explore what happens if we use Mean Squared Error (MSE) as our loss function. Please recall that the Mean Squared Error is a loss function typically used for regression tasks, but we are attempting to use it for a classification task. The largest problem when using MSE is that it can greatly depend on the scale of the data. 

In [ ]:
import torch
import pandas as pd
import numpy as np

np.random.seed(42)
torch.manual_seed(42)

from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import torch.nn.functional as F

# Parameters

In [ ]:
csv_file_prefix = 'spiral'
# csv_file_prefix = 'two_gaussians'
#csv_file_prefix = 'xor'
#csv_file_prefix = 'center_surround'

batch_size = 32
# train_fraction = 0.8

hidden_size = 64
lr=0.001
num_epochs = 1000

# Dataset

### NOTE TO SELF -- Are we supposed to fit_transform each dataset on its features, or only fit to the training data and transform train, test, and valid data based on training features?

In [ ]:
class SimpleDataset(torch.utils.data.Dataset):
    def __init__(self, csv_file, transform=None):
        super().__init__()
        data = pd.read_csv(csv_file)
        self.features = data.iloc[:, 1:]
        self.labels = data.iloc[:, 0]

        self.scaler = StandardScaler() # z-score all input features
        self.features = self.scaler.fit_transform(self.features)

        self.transform = transform

    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        feature = torch.tensor(self.features[idx], dtype=torch.float32)
        label = torch.tensor(self.labels[idx], dtype=torch.long)
        one_hot_label = F.one_hot(label, num_classes=2).float() # one-hot encoding the labels so we can perform MSELoss on this... tensor([0., 1.])) is 1 and tensor([1., 0.])) is 0
        return feature, one_hot_label

In [ ]:
train_dataset = SimpleDataset(csv_file=f'{csv_file_prefix}_train.csv')
valid_dataset = SimpleDataset(csv_file=f'{csv_file_prefix}_valid.csv')
test_dataset = SimpleDataset(csv_file=f'{csv_file_prefix}_test.csv')

In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Model

In [ ]:
class SimpleNN(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.f1 = torch.nn.Linear(input_size, hidden_size)
        self.relu = torch.nn.ReLU()
        self.f2 = torch.nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.f1(x)
        x = self.relu(x)
        x = self.f2(x)
        return x

In [ ]:
input_size = train_dataset.features.shape[1] 
output_size = len(set(train_dataset.labels))

In [ ]:
model = SimpleNN(input_size, hidden_size, output_size)
criterion = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9)

# Training the Model

In [ ]:
def train(model, train_loader, valid_loader, num_epochs, criterion, optimizer):
    train_losses = []
    valid_losses = []

    for epoch in range(num_epochs):
        model.train()
        running_train_loss = 0.0

        for features, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(features)
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

            running_train_loss += loss.item()

        epoch_train_loss = running_train_loss / len(train_loader)
        train_losses.append(epoch_train_loss)

        model.eval()
        running_valid_loss = 0.0

        with torch.no_grad():
            for features, labels in valid_loader:
                outputs = model(features)
                loss = criterion(outputs, labels)
                
                running_valid_loss += loss.item()

        epoch_valid_loss = running_valid_loss / len(valid_loader)
        valid_losses.append(epoch_valid_loss)

        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {epoch_train_loss:.4f}, Valid Loss: {epoch_valid_loss:.4f}")
        
    return train_losses, valid_losses

In [ ]:
train_losses, valid_losses = train(model, train_loader, valid_loader, num_epochs, criterion, optimizer)

# Testing the Model

In [ ]:
"""
outputs
tensor([[-0.0570,  1.0044],
        [-0.4251,  1.4154],
        [ 0.2088,  0.7987],

predicted
tensor([1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1,
        1, 1, 0, 0, 1, 1, 1, 1]) (NEED TO CONVERT INTO ONE_HOT)

total_test
32                
"""

def test(model, test_loader):
    model.eval()
    correct_test = 0
    total_test = 0
    with torch.no_grad():
        for features, one_hot_labels in test_loader:
                outputs = model(features)
                _, predicted = torch.max(outputs, 1)
                
                labels = torch.argmax(one_hot_labels, dim=1)

                total_test += labels.size(0)
                correct_test += (predicted == labels).sum().item()

    test_accuracy = 100 * correct_test / total_test
    print(f"Test Accuracy: {test_accuracy:.4f}")
    
    return test_accuracy

In [ ]:
test_accuracy = test(model, test_loader)

# LOSS CURVES

In [ ]:
plt.plot(list(range(1, (num_epochs) + 1)), train_losses, label='train loss')
plt.plot(list(range(1, (num_epochs) + 1)), valid_losses, label='valid loss')
plt.title(f'Loss Curves for the {csv_file_prefix.capitalize()} Dataset')
plt.xlabel('epoch')
plt.ylabel('cross entropy loss')
plt.legend()

plt.show()

# Visualizing Decision Boundaries

In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt

x_min, x_max = train_dataset.features[:, 0].min(), train_dataset.features[:, 0].max()
y_min, y_max = train_dataset.features[:, 1].min(), train_dataset.features[:, 1].max()

xx, yy = np.meshgrid(np.linspace(x_min, x_max, 100), np.linspace(y_min, y_max, 100))
grid_points = torch.tensor(np.c_[xx.ravel(), yy.ravel()], dtype=torch.float32)

with torch.no_grad():
    Z = model(grid_points)
    Z = Z.argmax(dim=1).numpy()  

Z = Z.reshape(xx.shape)

plt.contourf(xx, yy, Z, alpha=0.5, cmap='viridis')

for inputs, one_hot_labels in train_loader:
    inputs = inputs.numpy()
    labels = one_hot_labels.argmax(dim=1).numpy()  # CONVERTING ONE HOT BACK INTO LABEL
    plt.scatter(inputs[:, 0], inputs[:, 1], c=labels, edgecolor='k', cmap='coolwarm', marker='o')

plt.title("Decision Boundary with Training Data Points")
plt.xlabel("Feature 1")
plt.ylabel("Feature 2")
plt.show()
